In [20]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [21]:
from openai import OpenAI

client = OpenAI() # defautls to getting the key using os.environ.get("OPENAI_API_KEY")

In [22]:
resume_assistant = client.beta.assistants.retrieve("asst_fixO3zdAF2VSLVasrSpNMrDt")
print(resume_assistant)

Assistant(id='asst_fixO3zdAF2VSLVasrSpNMrDt', created_at=1720456903, description=None, instructions='You are a Resume Building Expert! \n\nThe user will input a job description. You will use the user\'s attached LinkedIn profile to build a resume that will give the user the best chance of getting the job described in the job description.\n\nThe created resume should be in JSON format, according to the following JSON Schema:\n{\n  "properties": {\n    "basics": {\n      "type": "object",\n      "additionalProperties": true,\n      "properties": {\n        "name": {\n          "type": "string"\n        },\n        "label": {\n          "type": "string",\n          "description": "e.g. Web Developer"\n        },\n        "image": {\n          "type": "string",\n          "description": "URL (as per RFC 3986) to a image in JPEG or PNG format"\n        },\n        "email": {\n          "type": "string",\n          "description": "e.g. thomas@gmail.com",\n          "format": "email"\n       

In [23]:
# Vector Store containing example linkedin profile
vector_store_files = client.beta.vector_stores.files.list(
  vector_store_id="vs_JjsnSAuBISfoU9EEmVZY43AX"
)
print(vector_store_files)

SyncCursorPage[VectorStoreFile](data=[VectorStoreFile(id='file-h2qquh1xpckfxGSuhGFRgvBv', created_at=1721953712, last_error=None, object='vector_store.file', status='completed', usage_bytes=6909, vector_store_id='vs_JjsnSAuBISfoU9EEmVZY43AX', chunking_strategy=ChunkingStrategyStatic(static=ChunkingStrategyStaticStatic(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))], object='list', first_id='file-h2qquh1xpckfxGSuhGFRgvBv', last_id='file-h2qquh1xpckfxGSuhGFRgvBv', has_more=False)


In [26]:
# Attach the vectorized example linkedIn profile to this thread
thread = client.beta.threads.create(
    tool_resources={
        'file_search': {
            'vector_store_ids': ["vs_JjsnSAuBISfoU9EEmVZY43AX"]
        }
    }
)
# read in contents of job description
with open("job_description.json", "r") as f:
    job_description = f.read()
    thread_message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=job_description,
    )